# Rheostat for Collaboration
In this notebook, I will outline a possible model that can be used in conjunction with others.<br>
7.10.2020<br>
Ankita Roychoudhury

In [2]:
# bioscrape
from bioscrape.types import Model
from bioscrape.simulator import py_simulate_model

#For arrays and plotting
import numpy as np
import pandas as pd

from scipy.integrate import simps
from numpy import trapz

# Import good plotting packages 
import bokeh.io
import bokeh.plotting
from bokeh.layouts import row
from bokeh.layouts import column
bokeh.io.output_notebook()

# from biocrnpyler.mechanism import Mechanism
from biocrnpyler.component import Component
# from biocrnpyler import Mixture
# from biocrnpyler.chemical_reaction_network import Species, Reaction, ComplexSpecies, ChemicalReactionNetwork
from biocrnpyler import *

Loading BokehJS ...

## Define Custom Mechanism, Component, and Mixture
Now, we will define a new function, Mechanism, Component, and Mixture to account for the desired energy use dynamics.The function (check_type) will check what type of item the species is and return the desired type. 
The Mechanisms will be called **FuelMichaelisMenten**, it will be of type 'catalysis,' it will take in an enzyme name and a _list_ of fuels, substrates, products, and wastes that are involved with that enzyme. It will consist of a binding, catalysis, and unbinding reaction, as shown in Model 1 above. We will create a Component called **Enzyme**. It will use mechanism 'catalysis' and will take in particular binding rates($ k_{uf}, k_{bf}, k_{cat}$). We will also define a new Mixture called **EnergyTxTl** that will use mechanism 'catalysis.'<br><br>


The Mechanism was implemented in BioCRNPyler as MichalisMentenReversible and the Component was implemented as MultiEnzyme. My code has minor adjustments (related to parameters, reversibility, and types). We will use my code in this notebook. (eventually will make the switch to be consistent)



### check_type function

In [3]:
def check_type(item, material_type_str): 
    if isinstance(item, Species):
        item_ret = item
    elif isinstance(item, str):
        item_ret = Species(name = item, material_type = material_type_str)
    elif isinstance(item, Component) and item.get_species() != None:
        item_ret = item.get_species()
    else:
        raise ValueError( item, "parameter must be a string, a Component with defined get_species(), or a chemical_reaction_network.species")
        
    return item_ret

### Mechanism: FuelMichaelisMenten

In [4]:
class FuelMichaelisMenten(Mechanism):
    def __init__(self, name, type = 'catalysis', **keywords):
        
        Mechanism.__init__(self = self, name = name, mechanism_type = type, **keywords)

    def update_species(self, enzyme, fuel_list, substrate_list, product_list, waste_list): 
        
        self.enzyme = check_type(enzyme, 'enzyme')
        
        species = [self.enzyme]
        comp1_list = [self.enzyme]
        comp2_list = [self.enzyme]
        
        for f in fuel_list:
            species.append(f)
            comp1_list.append(f)

        for s in substrate_list:
            species.append(s)
            comp1_list.append(s)
            
        for p in product_list:
            species.append(p)
            comp2_list.append(p)
            
        for w in waste_list:
            species.append(w)
            comp2_list.append(w)
               
        
        species += [ComplexSpecies(comp1_list)]
        species += [ComplexSpecies(comp2_list)]
        return species
    
    def update_reactions(self, enzyme, fuel_list, substrate_list, product_list, waste_list, k_bf, k_uf, k_cat, component = None,
                        part_id = None): 
        
        # Reverse binding rates
        k_br = 0.1*k_bf
        k_ur = 0.1*k_uf

        self.enzyme = check_type(enzyme, 'enzyme')
        
        # Define input lists
        comp1_list = [self.enzyme]
        comp2_list = [self.enzyme]
        
        for f in fuel_list:
            comp1_list.append(f)
            
        for s in substrate_list:
            comp1_list.append(s)

        for p in product_list:
            comp2_list.append(p)
            
        for w in waste_list:
            comp2_list.append(w)
        
        # Define Complexes
        comp1 = ComplexSpecies(comp1_list)
        comp2 = ComplexSpecies(comp2_list)
        
        # Define Reactions
        binding_rxn = Reaction(inputs = comp1_list, outputs=[comp1], k = k_bf, k_rev = k_br)
        cat_rxn = Reaction(inputs = [comp1], outputs = [comp2], k = k_cat)
        unbinding_rxn = Reaction(inputs = [comp2], outputs = comp2_list, k=k_uf, k_rev = k_ur)
        
        return [binding_rxn, cat_rxn, unbinding_rxn]

### Component: Enzyme

In [5]:
class Enzyme(Component):
    def __init__(self, enzyme_name, substrate, fuel, product, waste, k_bf, k_uf, k_cat = 36000/60, **keywords):
      
        # ENZYME NAME
        self.enzyme = check_type(enzyme_name, 'enzyme')
    
        # SUBSTRATE
        self.substrate_list = []
        for s in substrate:
            self.substrate_list.append(self.set_species(s, material_type = 'molecule'))
            
        # FUEL
        self.fuel_list = []
        for f in fuel:
            self.fuel_list.append(self.set_species(f, material_type ='metabolite'))
        
        
        # PRODUCT
        self.product_list = []
        for p in product:
            self.product_list.append(self.set_species(p, material_type = 'molecule'))
          
            
        # WASTE
        self.waste_list = []
        for w in waste:
            self.waste_list.append(self.set_species(w, material_type = 'metabolite'))
        
        
        self.k_bf = k_bf
        self.k_uf = k_uf
        self.k_cat = k_cat
      
        Component.__init__(self = self, name = enzyme_name, **keywords)
        
    def update_species(self):
        mech_cat = self.mechanisms['catalysis']

        return mech_cat.update_species(self.enzyme, self.fuel_list, self.substrate_list, self.product_list, self.waste_list) 
                                                                                           
    
    def update_reactions(self):
        mech_cat = self.mechanisms['catalysis']

        return mech_cat.update_reactions(self.enzyme, self.fuel_list, self.substrate_list, self.product_list, self.waste_list,self.k_bf,
                                         self.k_uf, self.k_cat,component = None,  part_id = None)
    

### Mixture: EnergyTxTl

In [6]:
class EnergyTxTl(Mixture):
    def __init__(self, name="",**keywords): 
        

        mech_cat = FuelMichaelisMenten('catalysis')
        
        default_mechanisms = {
            mech_cat.mechanism_type:mech_cat
        }
        
        Mixture.__init__(self, name = name, default_mechanisms=default_mechanisms, **keywords)     

## Entire pathway in BioCRNPyler
It is important that this pathway has a default ATP Leak reaction. This is designed to represent ATP use. If you are implementing this with your own model, you will want to remove this function. 


In [7]:
# Define rate constants
# k_bf = 15 # /hr
# k_uf = 15 # /hr

k_bf = 15/60
k_uf = 15/60 # /min
# Define reaction for each enzyme
E1_hex = Enzyme(enzyme_name = "hex", substrate = ['glucose'],
            fuel = ['atp'],product = ['g6p'], waste = ['adp'], k_bf = k_bf , k_uf = k_uf)

E2_pgi = Enzyme(enzyme_name = 'pgi', substrate = ['g6p'], fuel = [],
           product = ['f6p'], waste = [], k_bf = k_bf , k_uf = k_uf)

E3_pfk = Enzyme(enzyme_name = 'pfk', substrate = ['f6p'], fuel = ['atp'], product = ['f16p'],
           waste = ['adp'], k_bf = k_bf , k_uf = k_uf)

E4_ald_tpi = Enzyme(enzyme_name ='ald_tpi' , substrate = ['f16p'], fuel = [], product = ['g3p', 'g3p'], 
            waste = [], k_bf = k_bf , k_uf = k_uf )

E5_gapN = Enzyme(enzyme_name ='gapN' , substrate = ['g3p', 'g3p'], fuel = ['nadp', 'nadp'], product = ['3pg', '3pg'], 
            waste = ['nadph', 'nadph'], k_bf = k_bf , k_uf = k_uf)

E6_mGapDH = Enzyme(enzyme_name ='mGapDH' , substrate = ['g3p', 'g3p'], fuel = ['pi', 'nadp', 'nadp'], product = ['13bpg'], 
                waste = ['nadph', 'nadph'],k_bf = k_bf , k_uf = k_uf)

E7_pgk = Enzyme(enzyme_name = 'pgk', substrate = ['13bpg'], fuel = ['adp'], product = ['3pg', '3pg'], 
            waste = ['atp'], k_bf = k_bf , k_uf = k_uf)

E8_pgm = Enzyme(enzyme_name ='pgm' , substrate = ['3pg', '3pg'], fuel = [], product = ['2pg', '2pg'], 
            waste = [], k_bf = k_bf , k_uf = k_uf)

E9_eno = Enzyme(enzyme_name ='eno' , substrate = ['2pg', '2pg'], fuel = [], product = ['pep', 'pep'], 
            waste = [],k_bf = k_bf , k_uf = k_uf)

E10_pyk = Enzyme(enzyme_name = 'pyk', substrate = ['pep', 'pep'], fuel = ['adp', 'adp'], product = ['pyruvate', 'pyruvate'], 
            waste = ['atp', 'atp'], k_bf = k_bf , k_uf = k_uf) # irreversible

E11_alsS = Enzyme(enzyme_name = 'alsS', substrate = ['pyruvate', 'pyruvate'], fuel = [], product = ['acetolac'], 
            waste = [], k_bf = k_bf , k_uf = k_uf) # irreversible

E12_IlvC = Enzyme(enzyme_name = 'IlvC', substrate = ['acetolac'], fuel = ['nadph'], product = ['23dih3mebut'], 
            waste = ['nadp'], k_bf = k_bf , k_uf = k_uf)

E13_IlvD = Enzyme(enzyme_name ='IlvD' , substrate = ['23dih3mebut'], fuel = [], product = ['3me2oxo'], 
            waste = [],k_bf = k_bf , k_uf = k_uf)

E14_kivD = Enzyme(enzyme_name ='kivD' , substrate = ['3me2oxo'], fuel = [], product = ['isobutanal'], 
            waste = [], k_bf = k_bf , k_uf = k_uf) # irreversible

E15_yahk = Enzyme(enzyme_name = 'yahk', substrate = ['isobutanal'], fuel = ['nadph'], product = ['isobutanol'],
                  waste = ['nadp'], k_bf = k_bf , k_uf = k_uf)

# Define ATP Leak Only Enzyme
E16_all_other_atp = Enzyme(enzyme_name = 'atp_synthase', substrate = [], fuel = ['atp'], 
                           product = [], waste = ['adp', 'pi'], k_bf = k_bf, k_uf = k_uf, k_cat = 2/60,)

# Define the Mixture for Rheostat + ATP Leak
myMixture = EnergyTxTl(components = [E1_hex,E2_pgi,E3_pfk, E4_ald_tpi, E5_gapN, E6_mGapDH, E7_pgk, E8_pgm, E9_eno, E10_pyk, 
                                    E11_alsS, E12_IlvC, E13_IlvD, E14_kivD, E15_yahk, E16_all_other_atp])

myMixture = EnergyTxTl(components = [E1_hex,E2_pgi,E3_pfk, E4_ald_tpi, E5_gapN, E6_mGapDH, E7_pgk, E8_pgm, E9_eno, E10_pyk, 
                                    E11_alsS, E12_IlvC, E13_IlvD, E14_kivD, E15_yahk])

 
CRN = myMixture.compile_crn()

# Define Mixture for ATP Leak only
myMixture_atp = EnergyTxTl(components = [E16_all_other_atp])
CRN_atp = myMixture_atp.compile_crn()



Now, we will define timepoints, initial conditions, and perform the simulation.

In [12]:
# CRN.write_sbml_file("CRN.sbml")
# CRN_atp.write_sbml_file("CRN_atp.sbml")

# Define timepoints
timepoints = np.linspace(0,72*60,100)

# Define initial conditions for Rheostat + ATP Leak Model
e = 0.20
e4 = 2

x0 = {'molecule_glucose':30,
      'metabolite_atp': 5,
     'metabolite_nadp':30,
      'metabolite_pi':30,
      "enzyme_hex":e,
      'enzyme_pgi':e,
      'enzyme_pfk':e,
      'enzyme_ald_tpi':e,
      'enzyme_gapN':e, 
      'enzyme_mGapDH':e,
      'enzyme_pgk':e,
      'enzyme_pgm':e,
      'enzyme_eno':e,
      'enzyme_pyk':e,
      'enzyme_alsS':e,
      'enzyme_IlvC':e,
      'enzyme_IlvD':e,
      'enzyme_kivD':e,
      'enzyme_yahk':e,
#       "enzyme_atp_synthase":e4
}
     
# Define initial conditions for ATP Leak Model Only
x0_atp = {'metabolite_atp':5,
          "enzyme_atp_synthase":e4,
}

# Perform simulations
re = CRN.simulate_with_bioscrape(timepoints, initial_condition_dict = x0)
re_atp = CRN_atp.simulate_with_bioscrape(timepoints, initial_condition_dict = x0_atp)

Now, let's take a look at the plots!

In [13]:
# same color scheme as the others for presentation
colors=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02','#a6761d']
#colors = ['#8dd3c7','#ffffb3','#bebada','#fb8072','#80b1d3','#fdb462','#b3de69','#fccde5','#d9d9d9','#bc80bd','#ccebc5','#ffed6f']

# First plot glucose and isobutanol
p1_complex = bokeh.plotting.figure(width = 450, height = 250, 
                         x_axis_label = 'time',
                         y_axis_label = 'concentration (mM)',
                         title = 'Entire Pathway Biocrnpyler')
p1_complex.line(timepoints, re['molecule_glucose'], color = colors[0],line_width = 2, legend_label = 'glucose')
p1_complex.line(timepoints, re['molecule_isobutanol'], color = colors[1], line_width = 2,legend_label = 'isobutanol')
p1_complex.line(timepoints, re['molecule_f16p'], color = colors[2], line_width = 2,legend_label = 'f16p')
p1_complex.legend.location = 'center_right'
p1_complex.legend.click_policy="hide"


# Plot ATP and ADP, Pi
p2_complex = bokeh.plotting.figure(width = 450, height = 250,
                           x_axis_label = 'time (hrs)',
                         y_axis_label = 'concentration',
                                  title = 'ATP Compare')
p2_complex.line(timepoints, re['metabolite_atp'], color = colors[3], line_width = 2,legend_label = 'atp')
p2_complex.line(timepoints, re['metabolite_pi'], color = colors[6],line_width = 2, legend_label = 'pi')
p2_complex.line(timepoints, re['metabolite_adp'], color = colors[5], line_width = 2,legend_label = 'adp')
p2_complex.legend.location = 'center_right'
p2_complex.legend.click_policy="hide"


p2_complex.line(timepoints, re_atp['metabolite_atp'], legend_label = 'atp only',line_width=2, color = colors[4])
bokeh.io.show(row(p1_complex,p2_complex))

Cool! We see that the system with the rheostat generates more ATP!

In [13]:
#watermark
%reload_ext watermark
%watermark -v -p numpy,bokeh,jupyterlab,biocrnpyler,bioscrape

CPython 3.7.7
IPython 7.13.0

numpy 1.18.1
bokeh 2.0.2
jupyterlab 1.2.6
panel 0.9.5
biocrnpyler unknown
bioscrape 1.0.0
